Lakeroad / sam coward things
It is an interesting and possibly noble endeavor to try and make open hardware / fpga compiling as good as open software

Get out some intermediate rwepresentation from yosys.
Emulate stuff via iverilog
nmigen

https://github.com/YosysHQ/oss-cad-suite-build#installation
https://yowasp.org/ pip install yowasp-yosys

ffirtl
https://github.com/YosysHQ/picorv32
https://github.com/BrunoLevy/learn-fpga
femtorv quARK https://github.com/BrunoLevy/learn-fpga/blob/master/FemtoRV/RTL/PROCESSOR/femtorv32_quark.v


RTLIL is yosys internal IR. Text rep

Lakeroad https://arxiv.org/abs/2401.16526
https://github.com/uwsampl/lakeroad
https://github.com/uwsampl/lakeroad/blob/main/bin/verilog_to_racket.py yosys btor output


Could also root around in rachit's stuff

https://github.com/f4pga/f4pga-arch-defs

In [10]:
%%file /tmp/LUT2.v
// What does this do?
`default_nettype none
module LUT2(input A, B, output Z);
    parameter [3:0] init = 4'h0;
    wire [1:0] s1 = B ?     init[ 3:2] :     init[1:0];
    assign Z =      A ?          s1[1] :         s1[0];
endmodule

Overwriting /tmp/LUT2.v


In [6]:
%%file /tmp/yoscript.ys
read_verilog /tmp/LUT2.v

Overwriting /tmp/yoscript.ys


In [2]:
! yowasp-yosys -p read_verilog /tmp/LUT2.v #-s /tmp/yoscript.ys


 /----------------------------------------------------------------------------\
 |  yosys -- Yosys Open SYnthesis Suite                                       |
 |  Copyright (C) 2012 - 2024  Claire Xenia Wolf <claire@yosyshq.com>         |
 |  Distributed under an ISC-like license, type "license" to see terms        |
 \----------------------------------------------------------------------------/
 Yosys 0.42 (git sha1 9b6afcf3f, ccache clang 14.0.0-1ubuntu1.1 -Os -flto -flto)

-- Parsing `/tmp/LUT2.v' using frontend ` -vlog2k' --
ERROR: Can't open input file `/tmp/LUT2.v' for reading: No such file or directory


hmm. Looks like boolector or bitwuzla python bindings might support parsing btor2
https://bitwuzla.github.io/docs/python/interface.html#bitwuzla.Parser.parse
https://boolector.github.io/docs/boolector.html#pyboolector.Boolector.Parse

In [12]:
%%file /tmp/fifo.v

// address generator/counter
module addr_gen 
#(  parameter MAX_DATA=256,
	localparam AWIDTH = $clog2(MAX_DATA)
) ( input en, clk, rst,
	output reg [AWIDTH-1:0] addr
);
	initial addr <= 0;

	// async reset
	// increment address when enabled
	always @(posedge clk or posedge rst)
		if (rst)
			addr <= 0;
		else if (en) begin
			if (addr == MAX_DATA-1)
				addr <= 0;
			else
				addr <= addr + 1;
		end
endmodule //addr_gen

// Define our top level fifo entity
module fifo 
#(  parameter MAX_DATA=256,
	localparam AWIDTH = $clog2(MAX_DATA)
) ( input wen, ren, clk, rst,
	input [7:0] wdata,
	output reg [7:0] rdata,
	output reg [AWIDTH:0] count
);
	// fifo storage
	// sync read before write
	wire [AWIDTH-1:0] waddr, raddr;
	reg [7:0] data [MAX_DATA-1:0];
	always @(posedge clk) begin
		if (wen)
			data[waddr] <= wdata;
		rdata <= data[raddr];
	end // storage

	// addr_gen for both write and read addresses
	addr_gen #(.MAX_DATA(MAX_DATA))
	fifo_writer (
		.en     (wen),
		.clk    (clk),
		.rst    (rst),
		.addr   (waddr)
	);

	addr_gen #(.MAX_DATA(MAX_DATA))
	fifo_reader (
		.en     (ren),
		.clk    (clk),
		.rst    (rst),
		.addr   (raddr)
	);

	// status signals
	initial count <= 0;

	always @(posedge clk or posedge rst) begin
		if (rst)
			count <= 0;
		else if (wen && !ren)
			count <= count + 1;
		else if (ren && !wen)
			count <= count - 1;
	end

endmodule

Writing /tmp/fifo.v


In [2]:
! yosys /tmp/fifo.v

/bin/bash: line 1: yosys: command not found
